In [2]:
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import pandas as pd
import scipy.stats as stats
import scipy.sparse as sparse
import copy
import time
import warnings 
from scipy import integrate
import seaborn as sns
from numpy import sqrt, cosh, tanh, pi, exp
warnings.simplefilter('ignore')

import scipy.io as sio
import os
from tqdm.auto import tqdm

# using functions from PyalData
from pyaldata import *

## Load data

In [3]:
path = '/Users/Daphne/Data/'  # insert data path here
fname = 'Chewie_CO_VR_2016-10-06.mat' # insert filename here

mat = scipy.io.loadmat(path + fname, simplify_cells=True) # load matlab file 

In [4]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'trial_data'])

In [5]:
mat['__header__']

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Mar 11 17:00:03 2017'

In [6]:
mat['__globals__']

[]

In [7]:
mat['__version__']

'1.0'

In [8]:
mat['trial_data'][0]

{'monkey': 'Chewie',
 'date': '10-06-2016',
 'task': 'CO',
 'target_direction': -2.356194490192345,
 'trial_id': 1,
 'result': 'R',
 'bin_size': 0.01,
 'perturbation': 'VR',
 'perturbation_info': -0.52,
 'epoch': 'BL',
 'idx_trial_start': 11,
 'idx_target_on': 187,
 'idx_go_cue': 289,
 'idx_movement_on': 350,
 'idx_peak_speed': 364,
 'idx_trial_end': 437,
 'pos': array([[-5.39995089e+00, -2.51105898e+01],
        [-5.35244272e+00, -2.51461409e+01],
        [-5.30510903e+00, -2.51680017e+01],
        [-5.25463333e+00, -2.51901398e+01],
        [-5.19938582e+00, -2.52190694e+01],
        [-5.14155621e+00, -2.52482559e+01],
        [-5.08274793e+00, -2.52747392e+01],
        [-5.02262945e+00, -2.53021448e+01],
        [-4.96146807e+00, -2.53318134e+01],
        [-4.90057210e+00, -2.53608650e+01],
        [-4.84021080e+00, -2.53868913e+01],
        [-4.77947369e+00, -2.54097798e+01],
        [-4.71826269e+00, -2.54302298e+01],
        [-4.65753400e+00, -2.54489925e+01],
        [-4.5975698

In [10]:
# load into dataframe
df = mat2dataframe(path+fname, shift_idx_fields=True)

In [13]:
df.shape

(774, 24)

In [14]:
df.head()

monkey        date task  target_direction  trial_id result  bin_size  \
0  Chewie  10-06-2016   CO         -2.356194         1      R      0.01   
1  Chewie  10-06-2016   CO          1.570796         2      R      0.01   
2  Chewie  10-06-2016   CO          0.785398         3      R      0.01   
3  Chewie  10-06-2016   CO          3.141593         4      R      0.01   
4  Chewie  10-06-2016   CO         -1.570796         5      R      0.01   

  perturbation  perturbation_info epoch  ...  idx_peak_speed  idx_trial_end  \
0           VR              -0.52    BL  ...             363            436   
1           VR              -0.52    BL  ...             356            447   
2           VR              -0.52    BL  ...             414            483   
3           VR              -0.52    BL  ...             397            458   
4           VR              -0.52    BL  ...             301            362   

                                                 pos  \
0  [[-5.399950889827338, -25.11058978269873], [-5...   
1  [[-5.194667657852522, -35.190944353130526], [-...   
2  [[0.35267826312911055, -22.923278550949004], [...   
3  [[5.072834875801728, -24.89410841851521], [5.0...   
4  [[-8.194254800472459, -30.26359832666607], [-8...   

                                                 vel  \
0  [[4.615264447089089, -3.6781866137904946], [4....   
1  [[2.3921738136280783, 2.833707639241631], [2.7...   
2  [[0.5017102009485903, -3.146994710197926], [0....   
3  [[-1.3275097541588876, -3.5308863978243985], [...   
4  [[1.8501123719868446, 0.8394848146862552], [2....   

                                                 acc  \
0  [[34.913180411358354, -17.92603186704403], [13...   
1  [[31.99226756476989, 80.42711312188004], [34.1...   
2  [[-7.180978950112355, -35.21789901845524], [3....   
3  [[-6.932033881542791, 0.17982626642873387], [-...   
4  [[48.123169514922175, -9.081942493745938], [58...   

                                               force  \
0  [[0.3854857746577438, -0.18448026169727955], [...   
1  [[0.1778157378731141, 0.12542049016429982], [0...   
2  [[0.2833417683274762, -0.1285513884878812], [0...   
3  [[0.08675043530985374, 0.03658922221440793], [...   
4  [[0.2054977133197953, -0.12485484173352068], [...   

                                           M1_spikes  \
0  [[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,...   
1  [[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,...   
2  [[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...   
3  [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
4  [[0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,...   

                                       M1_unit_guide  \
0  [[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...   
1  [[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...   
2  [[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...   
3  [[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...   
4  [[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...   

                                          PMd_spikes  \
0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
1  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
2  [[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
3  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
4  [[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   

                                      PMd_unit_guide  
0  [[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2...  
1  [[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2...  
2  [[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2...  
3  [[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2...  
4  [[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2...  

[5 rows x 24 columns]